### Capstone 2- Bengali Grapheme Classification Project

This workbook is the first attempt at experimenting with a CNN for this project. I've followed this [starter code](https://www.kaggle.com/kaushal2896/bengali-graphemes-starter-eda-multi-output-cnn) for ideas. 

This is also the first attempt at using MLFlow for tracking deep learning experiments.

This set of experiments do not use image augmentation

In [1]:
#**********************************************************
#standard and ml imports
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from collections import defaultdict
import mlflow
import random
from keras import backend as K
K.tensorflow_backend._get_available_gpus() # ensuring keras uses GPU (rocket emoji)

#**********************************************************
# using custom scripts for creating model and loading data
import sys
sys.path.insert(1, './model/')

# helper function to automate model building
from model_creator import model_create
# helper function to import data
from data_loader import data_loader
# helper function to do image threshold filtering and resizing
from image_processor import image_processor_func
# helper function to start and track mlflow experiments
from experiments import run_experiment
# function to output final predictions on submission set
from tester import test_func
# function to do data augmentation for multiple output convnets - from this kernel 
# https://www.kaggle.com/kaushal2896/bengali-graphemes-starter-eda-multi-output-cnn
from batch_generator import MultiOutputDataGenerator

Using TensorFlow backend.


In [2]:
# defining function to reset graph and set seeds - from Aurelien Gueron's book HandsOn ML
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
reset_graph()

Before we begin our experiments, it is good practice to define an experiment matrix. We define the matrix with the following parameters :
- convolutional kernel size
- pooling size
- dropout rate (we assume both dropouts share the same rate)
- number of convolutional layers
- number of convolutional filters

The first experiment matrix contains a total of 108 experiments

In [4]:
# loading the pre-defined experiment csv
experiment_matrix = pd.read_csv('./experiment_matrix.csv',index_col='Experiment',)
experiment_matrix.head()

,kernel_size,pool_size,dropout,conv_layers,conv_filters
Experiment,,,,,
1,6,2,0.25,3,50
2,6,2,0.25,4,50
3,6,2,0.25,5,50
4,5,2,0.25,3,50
5,5,2,0.25,4,50


Next, we read the project csv files which define the labels

In [5]:
# read the project csv files
filenames = ['train','test','class_map','class_map_corrected','train_multi_diacritics','sample_submission']
df_dict = defaultdict()

for file in filenames:
    df_dict[file]=pd.read_csv('./data/{}.csv'.format(file))

Now, we start building our neural networks, start training and track training results using the MLFlow API. For this project, we use the MLFlow Keras Autologger

In [6]:
# define batch size and number of epochs for training
BATCH_SIZE = 100
EPOCHS = 50

In [7]:
# starting our experiments
import mlflow.keras
modelerrCount=0
experrCount=0
for index, experiment in experiment_matrix.iterrows():
    # resetting computational graph
    reset_graph()
    # specifying title
    title = 'exp_{0}_aug_no_kernel_size_{1}_pool_size_{2}_dropout_rate_{3}_conv_layers_{4}_conv_filters_{5}'.format(
    index,int(experiment.loc['kernel_size']),int(experiment.loc['pool_size']),experiment.loc['dropout'],int(experiment.loc['conv_layers']),
    int(experiment.loc['conv_filters']))
    try:
        print('Creating Model')
        # create model with early stopping enabled
        model,callbacks_list = model_create(input_shape=(68,118,1),conv_kernel_size=int(experiment.loc['kernel_size']),
                             pool_size=int(experiment.loc['pool_size']),
                            dropout_rate1=experiment.loc['dropout'],
                             dropout_rate2=experiment.loc['dropout'],
                            n_conv_layers=int(experiment.loc['conv_layers']),
                             conv_nfilters=int(experiment.loc['conv_filters']))
    except:
        # logging errors if any
        errCount+=1
        if errCount==1:
            with open('modelfailurelog.txt','w') as f:
                f.write('Experiment {} failed - Check Model Setup'.format(title))
        else:
            with open('modelfailurelog.txt','a') as f:
                f.write('Experiment {} failed - Check Model Setup'.format(title))
    print('Model Created')
    print(model.summary())
    print('Running Experiment-{}'.format(index))
    print("-------------------------------------")
    try:
        # running actual experiment
        run_experiment(title,model,callbacks_list,batch_size=BATCH_SIZE,epochs=EPOCHS,df_dict=df_dict,data_augmentation=False)
    except:
        # logging errors, if any
        experrCount+=1
        if experrCount==1:
            with open('experimentfailurelog.txt','w') as f:
                f.write('Experiment {} failed - Check Exp Setup'.format(title))
        else:
            with open('experimentfailurelog.txt','a') as f:
                f.write('Experiment {} failed - Check Exp Setup'.format(title))
print('All experiments finished')

Creating Model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model Created
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 68, 118, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 68, 118, 50)  500         input_layer[0][0]                
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 34, 59, 50)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 34, 59, 50)   22550       max_pooling2d_1

Model Created
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 68, 118, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 68, 118, 75)  2775        input_layer[0][0]                
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 34, 59, 75)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 34, 59, 75)   202575      max_pooling2d_1[0][0]            
______________________________________________________________________________

/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/mlflow/utils/autologging_utils.py:60: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_param_names, _, _, all_default_values = inspect.getargspec(fn)  # pylint: disable=W1505



Train on 45189 samples, validate on 5021 samples
Epoch 1/50
45189/45189 [==============================] - 49s 1ms/step - loss: 7.1923 - output_root_loss: 4.6637 - output_vowel_loss: 1.4422 - output_consonant_loss: 1.0859 - output_root_accuracy: 0.0461 - output_vowel_accuracy: 0.4999 - output_consonant_accuracy: 0.6335 - val_loss: 6.4546 - val_output_root_loss: 4.3203 - val_output_vowel_loss: 1.2451 - val_output_consonant_loss: 0.8749 - val_output_root_accuracy: 0.0661 - val_output_vowel_accuracy: 0.5845 - val_output_consonant_accuracy: 0.6873
Epoch 2/50
45189/45189 [==============================] - 40s 889us/step - loss: 3.7436 - output_root_loss: 2.7614 - output_vowel_loss: 0.4965 - output_consonant_loss: 0.4854 - output_root_accuracy: 0.2871 - output_vowel_accuracy: 0.8287 - output_consonant_accuracy: 0.8296 - val_loss: 3.1130 - val_output_root_loss: 2.2057 - val_output_vowel_loss: 0.4608 - val_output_consonant_loss: 0.4409 - val_output_root_accuracy: 0.4053 - val_output_vowel_acc

/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/mlflow/utils/autologging_utils.py:60: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_param_names, _, _, all_default_values = inspect.getargspec(fn)  # pylint: disable=W1505


...

45189/45189 [==============================] - 31s 689us/step - loss: 8.0366 - output_root_loss: 4.7019 - output_vowel_loss: 2.1282 - output_consonant_loss: 1.2066 - output_root_accuracy: 0.0274 - output_vowel_accuracy: 0.2017 - output_consonant_accuracy: 0.6204 - val_loss: 8.0063 - val_output_root_loss: 4.6923 - val_output_vowel_loss: 2.1102 - val_output_consonant_loss: 1.2023 - val_output_root_accuracy: 0.0257 - val_output_vowel_accuracy: 0.2145 - val_output_consonant_accuracy: 0.6198
Epoch 2/50
45189/45189 [==============================] - 31s 687us/step - loss: 8.0340 - output_root_loss: 4.6997 - output_vowel_loss: 2.1277 - output_consonant_loss: 1.2066 - output_root_accuracy: 0.0260 - output_vowel_accuracy: 0.2023 - output_consonant_accuracy: 0.6204 - val_loss: 8.0100 - val_output_root_loss: 4.6939 - val_output_vowel_loss: 2.1123 - val_output_consonant_loss: 1.2021 - val_output_root_accuracy: 0.0303 - val_output_vowel_accuracy: 0.1773 - val_output_consonant_accuracy: 0.6198
Epoch

/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/mlflow/utils/autologging_utils.py:60: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_param_names, _, _, all_default_values = inspect.getargspec(fn)  # pylint: disable=W1505


Epoch 1/50
45189/45189 [==============================] - 31s 690us/step - loss: 8.0220 - output_root_loss: 4.6941 - output_vowel_loss: 2.1327 - output_consonant_loss: 1.1952 - output_root_accuracy: 0.0266 - output_vowel_accuracy: 0.2065 - output_consonant_accuracy: 0.6259 - val_loss: 7.9967 - val_output_root_loss: 4.6811 - val_output_vowel_loss: 2.1148 - val_output_consonant_loss: 1.2029 - val_output_root_accuracy: 0.0277 - val_output_vowel_accuracy: 0.2043 - val_output_consonant_accuracy: 0.6218
Epoch 2/50
45189/45189 [==============================] - 31s 688us/step - loss: 8.0208 - output_root_loss: 4.6924 - output_vowel_loss: 2.1335 - output_consonant_loss: 1.1950 - output_root_accuracy: 0.0274 - output_vowel_accuracy: 0.2075 - output_consonant_accuracy: 0.6259 - val_loss: 8.0006 - val_output_root_loss: 4.6843 - val_output_vowel_loss: 2.1157 - val_output_consonant_loss: 1.2028 - val_output_root_accuracy: 0.0293 - val_output_vowel_accuracy: 0.2043 - val_output_consonant_accuracy: 0

/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/mlflow/utils/autologging_utils.py:60: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_param_names, _, _, all_default_values = inspect.getargspec(fn)  # pylint: disable=W1505


Epoch 1/50
45189/45189 [==============================] - 31s 689us/step - loss: 8.0273 - output_root_loss: 4.6955 - output_vowel_loss: 2.1353 - output_consonant_loss: 1.1966 - output_root_accuracy: 0.0282 - output_vowel_accuracy: 0.2057 - output_consonant_accuracy: 0.6236 - val_loss: 8.0181 - val_output_root_loss: 4.7022 - val_output_vowel_loss: 2.1188 - val_output_consonant_loss: 1.1929 - val_output_root_accuracy: 0.0257 - val_output_vowel_accuracy: 0.1980 - val_output_consonant_accuracy: 0.6284
Epoch 2/50
45189/45189 [==============================] - 31s 689us/step - loss: 8.0264 - output_root_loss: 4.6952 - output_vowel_loss: 2.1348 - output_consonant_loss: 1.1963 - output_root_accuracy: 0.0283 - output_vowel_accuracy: 0.2050 - output_consonant_accuracy: 0.6236 - val_loss: 8.0211 - val_output_root_loss: 4.7036 - val_output_vowel_loss: 2.1207 - val_output_consonant_loss: 1.1927 - val_output_root_accuracy: 0.0261 - val_output_vowel_accuracy: 0.1980 - val_output_consonant_accuracy: 0